# Convex Hulls

General information

## Table of Contents

1. Introduction
2. Imports
3. Algorithms
    * Naive Hull
    * Graham Scan
    * Gift Wrapping
    * Chan's Hull
4. Robustness
5. Experiments
6. Conclusion

## 2. Imports

In [ ]:
from enum import Enum, auto
import math


class Or(Enum):
    LEFT = auto()
    ON_BEFORE = auto()
    ON_BETWEEN = auto()
    ON_BEHIND = auto()
    RIGHT = auto()

class Point:
    def __init__(self,x,y):
        self.x = x
        self.y = y
        
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y
    
    def __hash__(self):
        return hash((self.x, self.y))
    
    def __repr__(self):
        return f"Point({self.x}, {self.y})"
    
    #def as_np(self):
    #    return np.array([self.x, self.y])
    
    def distance(self, other: 'Point') -> float:
        return math.sqrt((self.x - other.x)**2 + (self.y - other.y)**2)
    
    def orientation(self, p: 'Point', q: 'Point') -> Or:
        if p == q:
            raise ValueError("Line (segment) needs two different points")
        val = (q.x - p.x) * (self.y - p.y) - (q.y - p.y) * (self.x - p.x)
        if val > 0.0:
            return Or.LEFT
        elif val < 0.0:
            return Or.RIGHT
        else:
            if p.x != q.x:
                param = (self.x - p.x) / (q.x - p.x)
            else:
                param = (self.y - p.y) / (q.y - p.y)
            if param < 0.0:
                return Or.ON_BEFORE
            elif param > 1.0:
                return Or.ON_BEHIND
            else:
                return Or.ON_BETWEEN
            
class PointRef(Point):
    def __init__(self, container: list[Point], pos: int):
        self.container = container
        self.pos = pos
        
    def get(self) -> Point:
        return self.container[self.pos]
    
    @property
    def x(self):
        return self.get().x
    
    @property
    def y(self):
        return self.get().y
    
    def next_ref_in_container(self) -> 'PointRef':
        return PointRef(self.container, (self.pos+1) % len(self.container))
    
    def is_in_container(self, container: list[Point]) -> bool:
        return container is self.container
    
    #def __eq__(self, other):
    #    return self.x == other.x and self.y == other.y

In [ ]:
from ipycanvas import Canvas
canvas = Canvas(width=400, height=250)

canvas.fill_style = 'blue'
canvas.stroke_style = 'purple'
canvas.line_width = 5

## 3. Algorithms

### 3.1 Naive Hull
The first algorithm from the lecture

In [ ]:
def naive_hull(points: list[Point]) -> list[Point]:
    edges: dict[Point, Point] = {}
    for p in points:
        for q in filter(lambda q: q != p, points):
            valid = True
            for r in filter(lambda r: r != p and r != q, points):
                orient = r.orientation(p, q)
                if orient is Or.RIGHT or orient is Or.ON_BEFORE or orient is Or.ON_BEHIND:
                    valid = False
                    break
            if valid:
                edges[p] = q
    hull: list[Point] = []
    if edges:
        first_point, next_point = edges.popitem()
        hull.append(first_point)
        while next_point is not None and next_point != first_point:
            hull.append(next_point)
            next_point = edges.pop(next_point, None)
    return hull

In [ ]:
points = [Point(0,0), Point(100,50), Point(250,25), Point(100,100), Point(200,200), Point(300,0), Point(400,0)]
hull = naive_hull(points)
print(hull)

In [ ]:
canvas.fill_polygon(list(map(lambda p: (p.x, 250 - p.y), hull)))
canvas.stroke_polygon(list(map(lambda p: (p.x, 250 - p.y), hull)))

canvas.fill_style = "orange"
canvas.fill_circles(list(map(lambda p: p.x, points)), list(map(lambda p: 250 - p.y, points)), [5,5,5,5,5,5])

canvas

should handle degeneracies (sure?), but it's not robust

### 3.2 Graham Scan
Better algorithm

In [ ]:
def graham_scan(points: list[Point]) -> list[Point]:    
    if len(points) <= 2:
        return points[:]
    sorted_points = sorted(points, key = lambda p: (p.x, p.y))
    upper_hull = graham_half_scan(sorted_points)
    sorted_points.reverse()
    lower_hull = graham_half_scan(sorted_points)
    hull = upper_hull[:-1] + lower_hull[:-1]
    return hull

def graham_half_scan(points: list[Point]) -> list[Point]:
    if len(points) <= 2:
        return points[:]
    hull = points[:2]
    for p in points[2:]:
        hull.append(p)
        while len(hull) > 2 and hull[-2].orientation(hull[-3], hull[-1]) is not Or.LEFT:
            del hull[-2]
    return hull

In [ ]:
print(graham_scan([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(1,1), Point(2,2), Point(3,0), Point(4,0)]))

should handle degeneracies and be somewhat robust

### 3.3 Gift Wrapping
Also called *Jarvis's March*. Output-sensitive.

In [ ]:
def gift_wrapping(points: list[Point]) -> list[Point]:
    #if len(points) <= 2: // len(set(points)) <= 2:     # wie umgehen mit Duplikaten ? evtl. set nehmen...
    #    return points[:]
    first_point = max(points, key = lambda p: (p.x, p.y))
    wrapper = Wrapper(first_point)
    hull = [first_point]
    while True:
        next_point = wrapper.get_best_point(points)
        if next_point == first_point:
            break             # sicherstellen, dass dieser Punkt erreicht wird
        hull.append(next_point)
    return hull


class Wrapper:
    def __init__(self, last_point: Point):
        self.last_point = last_point
    
    def get_best_point(self, points: list[Point]) -> Point:
        points_iter = filter(lambda p: p != self.last_point, points)
        best_point = next(points_iter)
        for p in points_iter:
            if p != best_point and self.is_better(p, best_point):         # wie umgehen mit Duplikaten ?
                best_point = p
        self.last_point = best_point
        return best_point

    def is_better(self, p: Point, q: Point) -> bool:
        orient = p.orientation(self.last_point, q)
        return orient is Or.LEFT or orient is Or.ON_BEHIND

In [ ]:
print(gift_wrapping([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(1,1), Point(2,2), Point(3,0), Point(4,0)]))

### 3.4 Chan's Hull
Best of both worlds

In [ ]:
from typing import Optional
    
    
class ChanWrapper(Wrapper):
    def __init__(self, last_point: PointRef):
        super(ChanWrapper, self).__init__(last_point)
    
    def has_same_angle(self, p: Point, q: Point) -> bool:
        orient = p.orientation(self.last_point, q)
        return orient is Or.ON_BEHIND or orient is Or.ON_BETWEEN
    
    # Handles all the degenerate edge cases. Don't read it, lest it will haunt you in your nightmares.
    def calculate_indexes(self, mini_hull: list[Point], l: int, r: int) -> tuple[int, int, int, int, int]:
        if r - l < 2:
            return l, 0, 0, 0, r
        if self.has_same_angle(mini_hull[l], mini_hull[r]):
            if self.is_better(mini_hull[r], mini_hull[l]):
                l += 1
            else:
                r -= 1
            if r - l < 2:
                return l, 0, 0, 0, r
        m = int(l + (r - l) / 2)
        mb = m-1
        ma = m+1
        if self.has_same_angle(mini_hull[m], mini_hull[mb]):
            if self.is_better(mini_hull[mb], mini_hull[m]):
                m -= 1
            mb -= 1
        elif self.has_same_angle(mini_hull[m], mini_hull[ma]):
            if self.is_better(mini_hull[ma], mini_hull[m]):
                m += 1
            ma += 1
        return l, mb, m, ma, r
    
    # Can return more than one candidate but at most five which is asymptotically constant.
    def get_candidates(self, mini_hull: list[Point]) -> list[PointRef]:
        if self.last_point.is_in_container(mini_hull):
            return [self.last_point.next_ref_in_container()]
        l, mb, m, ma, r = self.calculate_indexes(mini_hull, 0, len(mini_hull) - 1)
        while l <= mb < m < ma <= r:
            m_better_than_before = self.is_better(mini_hull[m], mini_hull[mb])
            m_better_than_after = self.is_better(mini_hull[m], mini_hull[ma])
            if m_better_than_before and m_better_than_after:
                return [PointRef(mini_mull, m)]
            m_better_than_l = self.is_better(mini_hull[m], mini_hull[l])
            r_better_than_l = self.is_better(mini_hull[r], mini_hull[l])
            if r_better_than_l:
                if m_better_than_l and not m_better_than_before:
                    r = mb
                else:
                    l = m
            else:
                if m_better_than_l and m_better_than_before:
                    l = ma
                else:
                    r = m
            l, mb, m, ma, r = self.calculate_indexes(mini_hull, l, r)
        return [PointRef(mini_hull, pos) for pos in range(l, r+1)]

    
def chans_hull_m(points: list[Point], m: int) -> Optional[list[Point]]:
    mini_hulls = []
    mini_hull_points = []
    for i in range(0, len(points), m):
        j = min(len(points), i+m)
        mini_hull = graham_scan(points[i:j])
        mini_hulls.append(mini_hull)
        for pos in range(0, len(mini_hull)):
            mini_hull_points.append(PointRef(mini_hull, pos))
    first_point = max(mini_hull_points, key = lambda p: (p.x, p.y))
    wrapper = ChanWrapper(first_point)
    hull = [first_point]
    for _ in range(m-1):
        candidates = []
        for mini_hull in mini_hulls:
            candidates.extend(wrapper.get_candidates(mini_hull))
        next_point = wrapper.get_best_point(candidates)
        if next_point == first_point:
            return hull
        hull.append(next_point)
    return None

def chans_hull(points: list[Point]) -> list[Point]:
    #if len(points) <= 2:
    #    return points[:]
    hull = None
    t = 0
    while hull is None:
        m = min(len(points), 2**(2**t))
        hull = chans_hull_m(points, m)
        t += 1
    return hull

In [ ]:
print(chans_hull([Point(0,0), Point(1,0.5), Point(2.5,0.25), Point(1,1), Point(2,2), Point(3,0), Point(4,0), Point(1,2), Point(3,1.5)]))